In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [ ]:
IMAGE_SIZE = 256
batch_size = 32
CHANNELS = 3
EPOCHS = 30


In [ ]:
# Load the dataset using image_dataset_from_directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "beansLeaf",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size
)
class_names = dataset.class_names

In [ ]:

# Convert dataset to NumPy arrays
images = []
labels = []
for image_batch, label_batch in dataset:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())
images = np.concatenate(images)
labels = np.concatenate(labels)


In [ ]:
# Create an instance of the ImageDataGenerator with desired augmentation options
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    vertical_flip=False,
    fill_mode='nearest'
)


In [ ]:

# Create the directory if it doesn't exist
augmented_images_dir = "augmented_images"
if not os.path.exists(augmented_images_dir):
    os.makedirs(augmented_images_dir)

In [ ]:
# Generate augmented images and save them to the directory
augmented_data_generator = data_augmentation.flow(images, labels, batch_size=batch_size, shuffle=True)
for i, (augmented_images, augmented_labels) in enumerate(augmented_data_generator):
    for j, augmented_image in enumerate(augmented_images):
        save_path = os.path.join(augmented_images_dir, f"augmented_image_{i * batch_size + j}.png")
        tf.keras.preprocessing.image.save_img(save_path, augmented_image)
    if (i + 1) * batch_size >= len(images):
        break

In [ ]:

# Create a generator function for augmented image batches and labels
def augmented_data_generator():
    for image_batch, label_batch in data_augmentation.flow(images, labels, batch_size=batch_size, shuffle=False):
        yield image_batch, label_batch

In [ ]:

# Create a new dataset from the generator function
augmented_dataset = tf.data.Dataset.from_generator(
    augmented_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(batch_size,), dtype=tf.int32)
    )
)

In [ ]:
# Combine the original dataset with the augmented dataset
combined_dataset = dataset.concatenate(augmented_dataset)


In [ ]:
# Split the combined dataset into training and validation sets
val_size = int(0.2 * len(images))
train_dataset = combined_dataset.skip(val_size)
val_dataset = combined_dataset.take(val_size)

In [ ]:
# Creating a Layer for Resizing and Normalization
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])


In [ ]:

# Applying Data Augmentation to Train Dataset
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(class_names)

In [ ]:
# Load the pre-trained RNN model
rnn_model = keras.applications.MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet"
)


In [ ]:

# Freeze the layers of the pre-trained model
for layer in rnn_model.layers:
    layer.trainable = False

In [ ]:
# Create the CRNN model by adding RNN on top of the pre-trained model
model = models.Sequential([
    resize_and_rescale,
    rnn_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(n_classes, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [ ]:
# Train the CRNN model
history = model.fit(
    train_dataset,
    batch_size=batch_size,
    validation_data=val_dataset,
    verbose=1,
    epochs=EPOCHS
)

In [ ]:
# Evaluate the CRNN model on the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "beansLeaf/test",
    seed=123,
    shuffle=False,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size
)

In [ ]:
scores = model.evaluate(test_dataset)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
# Print the evaluation results
print("Test accuracy:", scores[1])

In [ ]:
# Plot the training and validation accuracy
plt.plot(acc)
plt.plot(val_acc)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation loss
plt.plot(loss)
plt.plot(val_loss)
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Make predictions on a sample image
test_images = []
test_labels = []
for image_batch, label_batch in test_dataset:
    test_images.append(image_batch.numpy())
    test_labels.append(label_batch.numpy())
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

for i in range(9):
    test_image = test_images[i]
    test_label = test_labels[i]
    predicted_class = class_names[np.argmax(model.predict(np.expand_dims(test_image, axis=0)))]
    actual_class = class_names[test_label]
    
    plt.imshow(test_image.astype("uint8"))
    plt.title(f"Actual: {actual_class}, Predicted: {predicted_class}")
    plt.axis("off")
    plt.show()